In [1]:
from datetime import date
import numpy as np
import pandas as pd
import ndjson
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from tqdm import tqdm

TOKENIZER = nltk.tokenize.word_tokenize

punctuation = "[{`,.?!:;/\()''""}¬]" #eventueel hier de hyphen weghalen zodat deze later gemerged kunnen worden in het outputfile


In [2]:
with open('../output/primitives_230807/primitives_corrected_monthly.ndjson') as fin:
    primitives = ndjson.load(fin)

df_primitives = pd.DataFrame(primitives)
df_primitives.shape

(101710, 6)

In [3]:
df_primitives.head()

,date,text,call_nr,id,clean_date,date_uncertainty
0,"[1768-07-03, 1782-06-xx, 1788-06-xx]",Traenotandum - De onder bovenstaande titel hie...,1795_Mege_Loef,1,1768-07-03,varying resolutions
1,[1768-07-03],1768 julii 3,1795_Mege_Loef,2,1768-07-03,unambiguous day
2,[1790-05-26],1790 maji 26 S. Sacramentum Confirmationis acc...,1795_Mege_Loef,4,1790-05-26,unambiguous day
3,[1789-11-xx],Ob Sanitatis insufficientiam hic Megae permans...,1795_Mege_Loef,14,1789-11-xx,unambiguous month
4,[1789-12-05],1789 dec. 5,1795_Mege_Loef,15,1789-12-05,unambiguous day


In [4]:
df_primitives['clean_month'] = df_primitives['clean_date'].str.replace('xx', '01')
df_primitives['clean_month'] = df_primitives['clean_month'].str.replace('XX', '01')
df_primitives.sort_values(by='clean_month', inplace=True)
df_primitives['len_text'] = df_primitives['text'].str.len()
df_primitives = df_primitives.query('len_text >= 50')
df_primitives.shape

(85680, 8)

In [5]:
all_rows = []

for index, line in tqdm(df_primitives.iterrows(), total=df_primitives.shape[0]):
    d_line = dict(line)
    d_line['n_tokens'] = len(TOKENIZER(''.join([c for c in d_line['text'] if c not in punctuation])))
    all_rows.append(d_line)

df_primitives = pd.DataFrame(all_rows)


100%|██████████| 85680/85680 [00:43<00:00, 1980.03it/s]


In [6]:
df_primitives['call_nr_clean'] = df_primitives['call_nr'].str[:14]
df_primitives['call_nr_clean'].nunique()

127

In [7]:
all_rows = []
false_rows = []


for i, r in df_primitives.iterrows():
    d_row = dict(r)
    try:
        d_row['clean_month'] = date.fromisoformat(d_row['clean_month'])
        all_rows.append(d_row)
    except ValueError:
        false_rows.append(d_row)

false_primitives = pd.DataFrame(false_rows)
false_primitives.shape

(35, 10)

In [8]:
false_primitives['call_nr'].unique()

array(['1657_Mech_Beth', '1838_Hoor_Anon_01', '1666_Gent_Bill_05',
       '1762_Maas_Anon', '1750_Brus_Anon', '1790_Mech_Wijn',
       '1772_Amst_Bikk', '1781_Brug_Ledo', '1792_Brie_Klui_09',
       '1776_Mech_Nieu_05', '1795_Maas_Loye', '1807_Leuv_Pelc_02',
       '1792_Brie_Klui_11', '1796_Leeu_Buma', '1807_Leuv_Pelc_03',
       '1808_Gent_Quic', '1805_Haar_Vinn', '1802_Brus_Anon',
       '1819_Binn_Kars', '1835_Gent_Call_02', '1835_Gent_Call_03',
       '1835_Gent_Call_05', '1854_Tilb_Beer_01', '1844_Heez_Moor',
       '1835_Gent_Call_06', '1855_Popp_Jong', '1856_Wird_Hell'],
      dtype=object)

In [9]:
with open('../output/primitives_230807/primitives_corrected_monthly_clean.ndjson', 'w') as fout:
        ndjson.dump(df_primitives.to_dict('records'), fout)